#### Copyright 2020 Google LLC.

In [25]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multiclass Classification

We previously created a binary classification model that determined if a piece of fruit was an orange or a grapefruit. There are many problems where binary classification can provide impactful solutions: spam or not spam in an email classifier, hit or hold in a blackjack simulation, buy or not in a stock market analysis. The list is basically endless.

There are other cases, however, where we want to make a decision across three or more classes. This is multiclass classification.

For many applications, multiclass classification can be broken down into many binary classification problems. These models employ a one-vs-all or one-vs-one strategy to create many binary classification tasks that are then aggregated into a multiclass classification model. Neural networks, decision trees, and k-nearest neighbors models are all capable of performing multiclass classification directly.

## The Dataset

For this unit we are going to use a classic machine learning dataset, the [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). This is a dataset that was used in 1936 by British biologist and statistician Ronald Fisher to classify iris flowers into one of three species based on four measurements:

- The length of the petals
- The width of the petals
- The length of the sepals (the green petal-looking bits that are found at the base of the petals)
- The width of the sepals

Conveniently, the iris dataset is built into the scikit-learn library, so it is readily available to us. Let's take a look:

In [26]:
from sklearn import datasets

iris_bunch = datasets.load_iris()
iris_bunch.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

Scikit-learn datasets are usually delivered in the form of a dictionary-like object called a `Bunch`. This `Bunch` contains the following fields:

- *DESCR*: A string describing the dataset.
- *data*: An array containing the features we are using for classifying. In this case, it's the four measurements listed above for each of 150 plants.
- *feature_names*: Labels for the data.
- *filename*: the file that this data came from.
- *target*: the values that we are trying to classify these flowers into. In this case, since we are dealing with three species of iris, we use three numbers (0, 1 and 2) to identify each species.
- *target_names*: labels for the target values. In this case, 0 refers to the setosa species, 1 to the versicolor species, and 2 to the virginica species.

We'll create a list of columns that we'll use for our model.

In [27]:
FEATURES = iris_bunch['feature_names']
TARGET = 'species'

FEATURES, TARGET

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 'species')

Next we will load the feature and target data into a Pandas dataframe.

In [28]:
import pandas as pd

iris_df = pd.DataFrame(iris_bunch['data'], columns=FEATURES)
iris_df[TARGET] = iris_bunch['target']

iris_df.sample(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
26,5.0,3.4,1.6,0.4,0
53,5.5,2.3,4.0,1.3,1
82,5.8,2.7,3.9,1.2,1
31,5.4,3.4,1.5,0.4,0
91,6.1,3.0,4.6,1.4,1
32,5.2,4.1,1.5,0.1,0
12,4.8,3.0,1.4,0.1,0
14,5.8,4.0,1.2,0.2,0
78,6.0,2.9,4.5,1.5,1
85,6.0,3.4,4.5,1.6,1


Let's take a look at a description of the data.

In [29]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


There are 150 data points. No columns seem to be missing data and no values seem to be too far out of expected ranges. For example, there are no zero or negative lengths or widths, and the length and width values fall well within what we'd expect for a tulip.

We are interested in using the measurement features to predict the species of an iris. Let's take a closer look at the values we'll be predicting.

In this case we'll group by our 'species' feature and get a count of each species in our dataset.

In [30]:
iris_df.groupby(TARGET).agg('count')

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
species,,,,
0,50,50,50,50
1,50,50,50,50
2,50,50,50,50


We have 50 examples of each species of iris, and overall we have only 150 samples. This presents two challenges. First, we don't have much data to actually build a model from. Second, the data that we do have is evenly distributed over class types. We might want to make sure that we train over the same distribution.

Luckily, there are solutions to both of these issues!

When we have data that has some weighted distribution across classes, we can do a **stratified split** to ensure that every class appears proportionally in our training data.

When we don't have enough data to properly train a model and don't feel that we can pull training data away, we can do a **k-fold cross validation** in order to utilize all of our data for training, while still trying to minimize model overfitting.

## Stratified Split

Let's first split off a set of data to use for our final model testing. We can use scikit-learn's `train_test_split` function to do this.

Since we have so little data, we'll only hold out 10% of the data for the final test.

After we make the split, we can see how many data points we will train off of for each class.

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    iris_df[FEATURES],
    iris_df[TARGET],
    test_size=0.1,
    random_state=45)

y_train.groupby(y_train).count()

species
0    43
1    50
2    42
Name: species, dtype: int64

Yikes! We kept 50 data points for training class 1. That means we left none for final testing:

In [32]:
y_test.groupby(y_test).count()

species
0    7
2    8
Name: species, dtype: int64

### Exercise 1: Stratified Train Test Split

We risk not holding out a data point for every class if we don't stratify our train test split. Rewrite the split above to create a stratified split. (If you don't remember how, try looking at the [documentation for `train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) and finding the argument that can be used to stratify the data.) When you are done, there should be 45 data points for each class in the training data and five data points for each class in the testing data. Print the counts to verify.

#### **Student Solution**

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    iris_df[FEATURES],
    iris_df[TARGET],
    test_size=0.1,
    stratify=iris_df[TARGET],
    random_state=45)

y_train.groupby(y_train).count()

species
0    45
1    45
2    45
Name: species, dtype: int64

---

## Cross-Validation

Another problem we have is that we have very little data to work with. We only had 150 data points in total and are only going to train using 135 of those data points. If we are going to be hyperparameter tuning, we'll need a test and validation holdout, which will leave us very little data to train on.

One way to get around this is to use cross-validation. Cross-validation splits the data into a fixed number of tranches and trains on `n-1` of the tranches. Then it calculates a score using the holdout tranche. It does this repeatedly, holding out one tranche of data for each training pass. By looking at the mean of the scores for each training pass, you can get an idea of how well your model performs without having to specify a test dataset.

The `cross_val_score` method is used to perform the cross-validation. In the example below, we divide the data into five tranches and get five scores.

Since we are cross-validating with a classifier, scikit-learn automatically performs stratified splits for us.

In [34]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

estimator = SGDClassifier()

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5
)

scores

array([0.44444444, 0.7037037 , 0.92592593, 0.66666667, 0.88888889])

We can now find the mean score.

In [35]:
scores.mean()

0.7259259259259259

What does this score represent, though? It turns out that it uses the default scoring method for the classifier that we used. In this case we used the [`SGDClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html), which reports accuracy by default.

Also note that the estimator isn't trained after running cross-validation. You can run cross-validation to test different data preprocessing pipelines and hyperparameters. Once you are happy with a specific setup, you'll need to train the model with the chosen pipeline and parameters.

### Exercise 2: F1 Scoring

What if we wanted to use F1 for our scoring metric instead of accuracy?

Run `cross_val_score` on an `SGDClassifier` and get the F1 score. Check out the documentation for [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [`make_scorer`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html), and [`f1_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) for clues.

#### **Student Solution**

In [36]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

estimator = SGDClassifier()

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
    scoring=make_scorer(f1_score, average='weighted')
)

scores.mean()

0.7575829725829727

---

# The Model Pipeline

Since we are now using cross-validation to train the model, we can use our testing holdout data as a final validation. Let's make that clear by renaming the data.

In [37]:
X_validation = X_test
y_validation = y_test

Now we can work on tuning the model and the model pipeline.

Let's first look back at the data going into the model:



In [38]:
iris_df[FEATURES].describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


The data is all in the same order of magnitude, but columns like 'sepal length (cm)' are considerably larger than columns like 'petal width (cm)'.

We need to perform some preprocessing to get the data into a more uniform shape before feeding it to the model. To do that we'll use the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), which removes the mean and subtracts the unit variance from each column of data.

To use the `StandardScaler`, we create the object, `fit()` the data, and then `transform()`.

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(iris_df[FEATURES])

pd.DataFrame(
    scaler.transform(iris_df[FEATURES]),
    columns=FEATURES
).describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02
mean,-1.690315e-15,-1.842970e-15,-1.698641e-15,-1.409243e-15
std,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00
min,-1.870024e+00,-2.433947e+00,-1.567576e+00,-1.447076e+00
25%,-9.006812e-01,-5.923730e-01,-1.226552e+00,-1.183812e+00
50%,-5.250608e-02,-1.319795e-01,3.364776e-01,1.325097e-01
75%,6.745011e-01,5.586108e-01,7.627583e-01,7.906707e-01
max,2.492019e+00,3.090775e+00,1.785832e+00,1.712096e+00


In [40]:
#My alternative approach: fit_transform is used instead of fit and transform
scaler = StandardScaler()

# scaler.fit(iris_df[FEATURES])

pd.DataFrame(
    scaler.fit_transform(iris_df[FEATURES]),
    columns=FEATURES
).describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02
mean,-1.690315e-15,-1.842970e-15,-1.698641e-15,-1.409243e-15
std,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00
min,-1.870024e+00,-2.433947e+00,-1.567576e+00,-1.447076e+00
25%,-9.006812e-01,-5.923730e-01,-1.226552e+00,-1.183812e+00
50%,-5.250608e-02,-1.319795e-01,3.364776e-01,1.325097e-01
75%,6.745011e-01,5.586108e-01,7.627583e-01,7.906707e-01
max,2.492019e+00,3.090775e+00,1.785832e+00,1.712096e+00


You can see in the output of `describe()` that the data now all has a standard deviation that approaches one.

We need to perform this preprocessing to features before training the model and before getting predictions. It can be error-prone to try to remember to do this. To make the task easier, we can create an estimator [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that applies our transformations and calls our estimator.

In [41]:
from sklearn.pipeline import Pipeline

estimator = Pipeline(
  steps=[
    ['scale', StandardScaler()],
    ['classifier', SGDClassifier()],
  ]
)

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
)

scores.mean()

0.9111111111111111

Scaling gave us a considerable jump in accuracy score. Hopefully you see similar results.

### Exercise 3: Final Validation

Our accuracy results were pretty good, so we aren't going to do any more hyperparameter tuning in this lab. Before we declare victory, though, we should find the F1 score of our validation data. Using our estimator pipeline, calculate the F1 score for `X_validation`.

In [42]:
scores = cross_val_score(
    estimator,
    X_validation[FEATURES],
    y_validation,
    cv=5,
    scoring=make_scorer(f1_score, average='weighted')
)

scores.mean()

0.7333333333333332

---

# Exercise 4: Winemaker Identification

Scikit-learn comes prepackaged with many toy datasets. These can be found in the [`sklearn.datasets` package](https://scikit-learn.org/stable/datasets/index.html). In this exercise we'll be working with the [wine dataset](https://scikit-learn.org/stable/datasets/index.html#wine-dataset).

The dataset contains information about the properties of wines produced by three different producers. The grapes that the producers used all come from the same region.

The columns are:

* alcohol
* malic_acid
* ash
* alcalinity_of_ash
* magnesium
* total_phenols
* flavanoids
* nonflavanoid_phenols
* proanthocyanins
* color_intensity
* hue
* od280/od315_of_diluted_wines
* proline

The target column is a 0, 1, or 2. Each number represents a different producer.

Your task in this exercise is to create a classifier that can identify the producer based on the wine properties.

Use as many code blocks as necessary to examine the data and build and validate your model. Document your process using text blocks and/or comments in your code.

**Student Solution**

In [43]:
# initial loading and set up
wine_bunch = datasets.load_wine()
FEATURES = wine_bunch['feature_names']
TARGET =  'target'

FEATURES, TARGET

(['alcohol',
  'malic_acid',
  'ash',
  'alcalinity_of_ash',
  'magnesium',
  'total_phenols',
  'flavanoids',
  'nonflavanoid_phenols',
  'proanthocyanins',
  'color_intensity',
  'hue',
  'od280/od315_of_diluted_wines',
  'proline'],
 'target')

In [44]:
# get FEATURES and TARGET into a dataframe
wine_df = pd.DataFrame(wine_bunch['data'], columns=FEATURES)
wine_df[TARGET] = wine_bunch['target']

print(wine_df.sample(10))
wine_df.describe()

     alcohol  malic_acid   ash  ...  od280/od315_of_diluted_wines  proline  target
102    12.34        2.45  2.46  ...                          3.38    438.0       1
114    12.08        1.39  2.50  ...                          3.19    385.0       1
34     13.51        1.80  2.65  ...                          2.87   1095.0       0
155    13.17        5.19  2.32  ...                          1.48    725.0       2
23     12.85        1.60  2.52  ...                          3.63   1015.0       0
100    12.08        2.08  1.70  ...                          2.96    710.0       1
80     12.00        0.92  2.00  ...                          3.12    278.0       1
150    13.50        3.12  2.62  ...                          1.30    500.0       2
143    13.62        4.95  2.35  ...                          2.05    550.0       2
165    13.73        4.36  2.26  ...                          1.75    520.0       2

[10 rows x 14 columns]


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


In [45]:
# check breakdown by target
wine_df.groupby(TARGET).agg('count')
# note {0:59, 1:71, 2:48}

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
target,,,,,,,,,,,,,
0,59,59,59,59,59,59,59,59,59,59,59,59,59
1,71,71,71,71,71,71,71,71,71,71,71,71,71
2,48,48,48,48,48,48,48,48,48,48,48,48,48


In [46]:
# split things up proportionally (stratify)
X_train, X_validation, y_train, y_validation = train_test_split(
    wine_df[FEATURES],
    wine_df[TARGET],
    test_size=0.1,
    stratify=wine_df[TARGET])

y_train.groupby(y_train).count()

target
0    53
1    64
2    43
Name: target, dtype: int64

In [47]:
# make the pipeline that will scale and classify
estimator = Pipeline(
  steps=[
    ['scale', StandardScaler()],
    ['classifier', SGDClassifier()],
  ]
)

# calculate accuracy
scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
)

# mean accuracy
scores.mean()

0.975

In [48]:
# calculate accuracy
scores = cross_val_score(
    estimator,
    X_validation[FEATURES],
    y_validation,
    cv=5,
)

# mean accuracy
scores.mean()

0.85

---